In [1]:
import numpy as np

In [2]:
class Perceptron:
    
    def __init__(self, learning_rate = 0.1):
        self._input_size = 0
        self._output_size = 0
        self._weights = None
        self.b = None
        self._num_samples = 0
        self._lr = learning_rate

    
    def fit(self, X, y, num_epochs = 100, batch_size=None, verbose=False):
        if type(X) is not np.ndarray:
            raise Exception('X deve ser uma matrix do tipo numpy.ndarray')

        if type(y) is not np.ndarray:
            raise Exception('y deve ser um vetor do tipo numpy.ndarray')
    
        
        # hot encode y caso ele não venha neste formato
        if len(y.shape) == 1: # se só for um vetor então o shape só vai retornar um valor
            old_y = y.copy()
            num_class = len(set(old_y))
            
            # transforma cada y em um vetor do tamanho do número de classes
            y = np.zeros((old_y.shape[0], 3))

            # seta para 1 as posições dos perceptrons correspondentes as classes
            for i in xrange(old_y.shape[0]):
                posi = old_y[i]
                y[i][posi] = 1
        
      
        # batch size é o número de elementos que serão usados em cada época pra atualizar o peso
        # caso não seja definido, vou utilizar todos os dados de treinamento disponíveis
        if batch_size is None:
            batch_size = X.shape[0]
        
        
        # armazenando parâmetros dos dados
        self._num_samples = X.shape[0]
        self._input_size  = X.shape[1]
        self._output_size = y.shape[1]
        
        # cada neurônio vai ter um conjunto de pesos igual ao número de entradas, 
        self._weights =  np.random.rand(self._output_size, self._input_size)
        
        # cada neurônio tem seu próprio bias
        self._b =  np.ones(self._output_size)
        
        
        # aqui inicializa o treinamento
        for i in range(num_epochs):

            # pego um conjunto de posições representando elemento da amostra igual ao tamanho do batch size
            posi_to_sample = np.random.randint(0, self._num_samples, batch_size)
            for posi in posi_to_sample:
                
                # obtendo um elemento específico                 
                x = X[posi]
                y_expected = y[posi]

                # calculando Z
                Z = np.dot(self._weights, x) + self._b

                #  calculando a saída dos neurônios
                y_hat = self._activation_function(Z)

                # calculando parte do gradiente            
                error = (y_hat  - y_expected)
                activation_derivative = self._activation_function_derivative(Z)
                delta =  error * activation_derivative # delta é só parte da derivada, vou gerar o gradiente completo abaixo
                                
                #  calculando o gradiente para cada neurônio e atualizando seus pesos
                for k in range(self._output_size):        
                    
                    gradient_w = (1.0/batch_size) * np.dot(delta[k] , x) # como troquei a minha função de erro pra erro médio, o gradiente fica com 1/n
                    gradient_b = (1.0/batch_size) * delta[k]

                    # finalmente atualizando os pesos
                    self._weights[k] = self._weights[k] - (self._lr * gradient_w)
                    self._b[k] = self._b[k] - (self._lr * gradient_b)
            
            
            # pra não imprimir sempre, vou mostrar acc a cada 10 iterações
            if verbose and (i + 1) % 10 == 0:
                print ("Epoch[%d]: acc: %f " %(i+1,  self._acc(X, y) ))
    
    
    def predict(self, X):
        # Z é um vetor, cada neurônio vai ter uma saida, portando, o shape de Z é igual a (num_exemplos, num_classes)
        Z = np.array([ np.dot(self._weights, x) + self._b  for x in X])
        
        # aplicando a função de ativação
        output = self._activation_function(Z)
        
        # como o y gerado é um vetor, estou retornando somente as classes relacionadas
        return np.argmax(output, axis=1)
        
    
    def _activation_function(self, Z):
        return 1.0 / (1.0 + np.e**(-Z))
    
    def _activation_function_derivative(self, Z):
        res_activation = self._activation_function(Z)
        return res_activation * (1 - res_activation) 
    
    
    def _acc(self, X, y):
        count = 0
        y_hat = self.predict(X)
        for i in range(len(y)):
            if np.argmax(y[i]) == y_hat[i]:
                count += 1

        return float(count) / len(y)
    
    

In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = iris.data
y = iris.target

# X = StandardScaler().fit_transform(X) # descomente esta linha pra melhorar o resultado, acredito que fique em média mais estável


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

percep = Perceptron(learning_rate=0.2)
percep.fit(X_train, y_train, num_epochs=1000, batch_size=32, verbose=True)

# avaliando com dados não vistos
print ("\n\n")
print ("#" * 30)
y_hat = percep.predict(X_test)
print ("ACC-TEST: ", accuracy_score(y_hat, y_test))
print ("#" * 30)
for i in range(len(y_hat)):
    print ("y_esperado: %d -- y_obtido: %d " %(y_test[i], y_hat[i]))





NameError: name 'xrange' is not defined